In [1]:
import pandas as pd
import seaborn as sns

In [2]:
df = pd.read_csv('results.csv') 

In [3]:
# compute score for each participant
for i in range(df.shape[1]):
    if df.iloc[:,i].name == "I daydream and fantasize, with some regularity, about things that might happen to me.":
        iri_start = i
        break

# The questions were sorted by the number presented in the paper.
iri_rules = {
    "0" :   ["Fantasy Scale", "+"],
    "1" :   ["Empathic Concern Scale", "+"],
    "2" :   ["Perspective-Taking Scale", "-"],
    "3" :   ["Empathic Concern Scale", "-"],
    "4" :   ["Fantasy Scale", "+"],
    "5" :   ["Fantasy Scale", "-"],
    "6" :   ["Perspective-Taking Scale", "+"],
    "7" :   ["Empathic Concern Scale", "+"],
    "8" :   ["Perspective-Taking Scale", "+"],
    "9" :   ["Fantasy Scale", "-"],
    "10":   ["Empathic Concern Scale", "-"],
    "11":   ["Perspective-Taking Scale", "-"],
    "12":   ["Fantasy Scale", "+"],
    "13":   ["Empathic Concern Scale", "-"],
    "14":   ["Empathic Concern Scale", "+"],
    "15":   ["Perspective-Taking Scale", "+"],
    "16":   ["Empathic Concern Scale", "+"],
    "17":   ["Fantasy Scale", "+"],
    "18":   ["Perspective-Taking Scale", "+"],
    "19":   ["Fantasy Scale", "+"],
    "20":   ["Perspective-Taking Scale", "+"]
}

df_iri = pd.DataFrame()


for id in df.index: # for each participant
    pt, f, e = 0, 0, 0 # perspective-taking, fantasy, empathic

    for i, x in enumerate(list(df.loc[id][iri_start:iri_start+21])):
        if iri_rules[str(i)][0] == "Fantasy Scale":
            f += x if iri_rules[str(i)][1] == "+" else -x
        elif iri_rules[str(i)][0] == "Empathic Concern Scale":
            e += x if iri_rules[str(i)][1] == "+" else -x
        elif iri_rules[str(i)][0] == "Perspective-Taking Scale":
            pt += x if iri_rules[str(i)][1] == "+" else -x

    pt, f, e = pt/7.0, f/7.0, e/7.0
    
    df_iri = df_iri.append(
        {
            "id" : id,
            "Perspective-Taking Scale" : pt,
            "Empathic Concern Scale" : e,
            "Fantasy Scale" : f
        },
        ignore_index=True
    )


df_iri.set_index("id", inplace=True)

df = df.merge(df_iri, left_index=True, right_index=True, how="outer")
df.drop(df.columns[iri_start:iri_start+21], axis=1, inplace=True)

In [4]:
df.to_csv("results.csv", sep=";")